In [ ]:
from collections import OrderedDict
from data_iterator import TextIterator
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import pickle as pkl
import numpy
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

num_words = 100140
batch_size=32
valid_batch_size=32
dim_word = 300
char_nout = 100
dim_char_emb = 15
learning_rate = 0.00001
dim_hidden = 600
max_len = 100

In [ ]:
temp = Variable(torch.LongTensor([[2,1],[3,4]]).cuda())
x= torch.zeros(3, 3)
x[:2,1]=1
x

In [ ]:
def str2list(s):
    alphabet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
    l = len(s)
    ans = []
    for i in range(0, l):
        a = alphabet.find(s[i])
        if a >= 0:
            ans.append(a)
        else:
            ans.append(0)
    return ans

In [ ]:
def prepare_data(seqs_x, seqs_y, labels, worddicts_r, maxlen=None):
    # x: a list of sentences
    lengths_x = [len(s) for s in seqs_x]
    lengths_y = [len(s) for s in seqs_y]

    if maxlen is not None:
        new_seqs_x = []
        new_seqs_y = []
        new_lengths_x = []
        new_lengths_y = []
        new_labels = []
        for l_x, s_x, l_y, s_y, l in zip(lengths_x, seqs_x, lengths_y, seqs_y, labels):
            if l_x < maxlen and l_y < maxlen:
                new_seqs_x.append(s_x)
                new_lengths_x.append(l_x)
                new_seqs_y.append(s_y)
                new_lengths_y.append(l_y)
                new_labels.append(l)
        lengths_x = new_lengths_x
        seqs_x = new_seqs_x
        lengths_y = new_lengths_y
        seqs_y = new_seqs_y
        labels = new_labels

        if len(lengths_x) < 1 or len(lengths_y) < 1:
            return None

    max_char_len_x = 0
    max_char_len_y = 0
    seqs_x_char = []
    l_seqs_x_char = []
    seqs_y_char = []
    l_seqs_y_char = []

    for idx, [s_x, s_y, s_l] in enumerate(zip(seqs_x, seqs_y, labels)):
        temp_seqs_x_char = []
        temp_l_seqs_x_char = []
        temp_seqs_y_char = []
        temp_l_seqs_y_char = []
        for w_x in s_x:
            word = worddicts_r[w_x]
            word_list = str2list(word)
            l_word_list = len(word_list)
            temp_seqs_x_char.append(word_list)
            temp_l_seqs_x_char.append(l_word_list)
            if l_word_list >= max_char_len_x:
                max_char_len_x = l_word_list
        for w_y in s_y:
            word = worddicts_r[w_y]
            word_list = str2list(word)
            l_word_list = len(word_list)
            temp_seqs_y_char.append(word_list)
            temp_l_seqs_y_char.append(l_word_list)
            if l_word_list >= max_char_len_y:
                max_char_len_y = l_word_list

        seqs_x_char.append(temp_seqs_x_char)
        l_seqs_x_char.append(temp_l_seqs_x_char)
        seqs_y_char.append(temp_seqs_y_char)
        l_seqs_y_char.append(temp_l_seqs_y_char)

    n_samples = len(seqs_x)
    maxlen_x = max(lengths_x)
    maxlen_y = max(lengths_y)
    if torch.cuda.is_available():
        x = torch.zeros(maxlen_x, n_samples).long().cuda()
        y = torch.zeros(maxlen_y, n_samples).long().cuda()
        x_mask = torch.zeros(maxlen_x, n_samples).cuda()
        y_mask = torch.zeros(maxlen_y, n_samples).cuda()
        l = torch.zeros(n_samples,).long().cuda()
        char_x = torch.zeros(maxlen_x, n_samples, max_char_len_x).long().cuda()
        char_x_mask = torch.zeros(maxlen_x, n_samples, max_char_len_x).cuda()
        char_y = torch.zeros(maxlen_y, n_samples, max_char_len_y).long().cuda()
        char_y_mask = torch.zeros(maxlen_y, n_samples, max_char_len_y).cuda()
    else:
        x = torch.zeros(maxlen_x, n_samples).long()
        y = torch.zeros(maxlen_y, n_samples).long()
        x_mask = torch.zeros(maxlen_x, n_samples)
        y_mask = torch.zeros(maxlen_y, n_samples)
        l = torch.zeros(n_samples,).long()
        char_x = torch.zeros(maxlen_x, n_samples, max_char_len_x).long()
        char_x_mask = torch.zeros(maxlen_x, n_samples, max_char_len_x)
        char_y = torch.zeros(maxlen_y, n_samples, max_char_len_y).long()
        char_y_mask = torch.zeros(maxlen_y, n_samples, max_char_len_y)

    for idx, [s_x, s_y, ll] in enumerate(zip(seqs_x, seqs_y, labels)):
        if torch.cuda.is_available():
            x[:lengths_x[idx], idx] = torch.Tensor(s_x).cuda()
            x_mask[:lengths_x[idx], idx]= 1.
            y[:lengths_y[idx], idx] = torch.Tensor(s_y).cuda()
            y_mask[:lengths_y[idx], idx] = 1.
            l[idx] = int(ll)

            for j in range(0, lengths_x[idx]):
                char_x[j, idx, :l_seqs_x_char[idx][j]] = torch.Tensor(seqs_x_char[idx][j]).cuda()
                char_x_mask[j, idx, :l_seqs_x_char[idx][j]] = 1.
            for j in range(0, lengths_y[idx]):
                char_y[j, idx, :l_seqs_y_char[idx][j]] = torch.Tensor(seqs_y_char[idx][j]).cuda()
                char_y_mask[j, idx, :l_seqs_y_char[idx][j]] = 1.
        else:
            x[:lengths_x[idx], idx] = torch.Tensor(s_x)
            x_mask[:lengths_x[idx], idx]= 1.
            y[:lengths_y[idx], idx] = torch.Tensor(s_y)
            y_mask[:lengths_y[idx], idx] = 1.
            l[idx] = int(ll)

            for j in range(0, lengths_x[idx]):
                char_x[j, idx, :l_seqs_x_char[idx][j]] = torch.Tensor(seqs_x_char[idx][j])
                char_x_mask[j, idx, :l_seqs_x_char[idx][j]] = 1.
            for j in range(0, lengths_y[idx]):
                char_y[j, idx, :l_seqs_y_char[idx][j]] = torch.Tensor(seqs_y_char[idx][j])
                char_y_mask[j, idx, :l_seqs_y_char[idx][j]] = 1.

    return Variable(x), Variable(x_mask), Variable(char_x), Variable(char_x_mask), Variable(y), Variable(y_mask), Variable(char_y), Variable(char_y_mask), Variable(l)

In [ ]:
class LSTM(nn.Module):
    def __init__(self, nin, hidden_size):
        super(LSTM, self).__init__()
        if torch.cuda.is_available():
            self.linear_f = nn.Linear(nin + hidden_size, hidden_size).cuda()
            self.linear_i = nn.Linear(nin + hidden_size, hidden_size).cuda()
            self.linear_ctilde = nn.Linear(nin + hidden_size, hidden_size).cuda()
            self.linear_o = nn.Linear(nin + hidden_size, hidden_size).cuda()
            
           
        else:     
            self.linear_f = nn.Linear(nin + hidden_size , hidden_size)
            self.linear_i = nn.Linear(nin + hidden_size, hidden_size)
            self.linear_ctilde = nn.Linear(nin + hidden_size, hidden_size)
            self.linear_o = nn.Linear(nin + hidden_size , hidden_size)
            

        self.hidden_size = hidden_size 
        self.init_weights()
    
    def forward(self, x, mask):
        #embs = torch.chunk(x, x.size()[1], 1)
        hidden, c = self.init_hidden(x.size(1))
        
        def step(emb, hid, c_t_old, mask_cur):
            combined = torch.cat((hid, emb), 1)
           
            f = F.sigmoid(self.linear_f(combined))
            i = F.sigmoid(self.linear_i(combined))
            o = F.sigmoid(self.linear_o(combined))
            c_tilde = F.tanh(self.linear_ctilde(combined))
            
            c_t = f * c_t_old + i * c_tilde
            c_t = mask_cur[:, None] * c_t + (1. - mask_cur)[:, None] * c_t_old
            
            hid_new = o * F.tanh(c_t)
            hid_new = mask_cur[:, None] * hid_new + (1. - mask_cur)[:, None] * hid
            
            return hid_new, c_t, i
        
        h_hist = []
        i_hist = []
        for i in range(x.size(0)):
            hidden, c, i = step(x[i].squeeze(), hidden, c, mask[i])     
            h_hist.append(hidden[None, :, :])
            i_hist.append(i[None, :, :])
            
        return torch.cat(h_hist), torch.cat(i_hist)

    def init_hidden(self, bat_size):
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(bat_size, self.hidden_size).cuda())
            c0 = Variable(torch.zeros(bat_size, self.hidden_size).cuda())
        else:
            h0 = Variable(torch.zeros(bat_size, self.hidden_size))
            c0 = Variable(torch.zeros(bat_size, self.hidden_size))
        return h0, c0
    
    def init_weights(self):
        initrange = 0.1
        lin_layers = [self.linear_f, self.linear_i, self.linear_ctilde, self.linear_o]
     
        for layer in lin_layers:
            layer.weight.data.uniform_(-initrange, initrange)
            layer.bias.data.fill_(0)


In [ ]:

class NLI(nn.Module):
    def __init__(self, dim_word, char_nout, dim_char_emb, word_embeddings_file, worddict, num_words, dim_hidden):
        super(NLI, self).__init__()
        self.dim_word = dim_word
        self.char_nout = char_nout
        self.dim_char_emb = dim_char_emb 
        self.char_k_cols = dim_char_emb
        self.char_k_rows=[1,3,5]
        self.hidden_size = dim_hidden
        self.word_embeddings = self.create_word_embeddings(word_embeddings_file, worddict, num_words, dim_word)
        
        dim_emb = dim_word + 3*char_nout
        self.alphabet = " abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"

        self.LSTM1 = LSTM(dim_emb, dim_hidden)
        self.LSTM1_rev = LSTM(dim_emb, dim_hidden)
        self.LSTM2 = LSTM(dim_emb+2*dim_hidden, dim_hidden)
        self.LSTM2_rev = LSTM(dim_emb+2*dim_hidden, dim_hidden)
        self.LSTM3 = LSTM(dim_emb+2*dim_hidden, dim_hidden)
        self.LSTM3_rev = LSTM(dim_emb+2*dim_hidden, dim_hidden)
        
        if torch.cuda.is_available():
            print('CUDA available')
            self.drop = nn.Dropout(p=0.5).cuda()
            self.conv1 = nn.Conv2d(1, self.char_nout, (self.char_k_rows[0], self.char_k_cols)).cuda()
            self.conv2 = nn.Conv2d(1, self.char_nout, (self.char_k_rows[1], self.char_k_cols)).cuda()
            self.conv3 = nn.Conv2d(1, self.char_nout, (self.char_k_rows[2], self.char_k_cols)).cuda()
            self.Charemb = nn.Embedding(len(self.alphabet) + 1, self.dim_char_emb, padding_idx=0).cuda()
            self.Linear1 = nn.Linear(24*dim_hidden, dim_hidden).cuda()
            self.Linear2 = nn.Linear(25*dim_hidden, dim_hidden).cuda()
            self.Linear3 = nn.Linear(dim_hidden, 3).cuda()
        else:
            self.Charemb = nn.Embedding(len(self.alphabet) + 1, self.dim_char_emb, padding_idx=0)
            self.Linear1 = nn.Linear(24*dim_hidden, dim_hidden)
            self.Linear2 = nn.Linear(25*dim_hidden, dim_hidden)
            self.Linear3 = nn.Linear(dim_hidden, 3)
            self.drop = nn.Dropout(p=0.5)
            self.conv1 = nn.Conv2d(1, self.char_nout, (self.char_k_rows[0], self.char_k_cols))
            self.conv2 = nn.Conv2d(1, self.char_nout, (self.char_k_rows[1], self.char_k_cols))
            self.conv3 = nn.Conv2d(1, self.char_nout, (self.char_k_rows[2], self.char_k_cols))

        
        self.init_weights(dim_hidden)

    def create_word_embeddings(self, file_name, worddicts, num_words, dim_word):
        if torch.cuda.is_available():
            word_embeddings = Variable(torch.zeros(num_words, dim_word).cuda())
        else:
            word_embeddings = Variable(torch.zeros(num_words, dim_word))
        word_embeddings.data.normal_(0, 0.01)
        with open(file_name, 'r') as f:
            for line in f:
                tmp = line.split()
                word = tmp[0]
                vector = tmp[1:]
                len_vec = len(vector)
                
                if(len_vec>300):
                    diff = len_vec-300
                    word = word.join(vector[:diff])
                    vector = vector[diff:]
                    
                    
                if word in worddicts and worddicts[word] < num_words:
                    vector = [float(x) for x in vector]
                    if torch.cuda.is_available():
                        word_embeddings[worddicts[word], :] = torch.FloatTensor(vector[0:300]).cuda()
                    else:
                        word_embeddings[worddicts[word], :] = torch.FloatTensor(vector[0:300])
            
        return word_embeddings
        
    def forward(self, premise, char_premise, premise_mask, char_premise_mask, hypothesis, char_hypothesis, hypothesis_mask, char_hypothesis_mask,l):
        #premise = number of words * number of samples. Also hypothesis = number of words * number of samples
        n_timesteps_premise = premise.size(0)
        n_timesteps_hypothesis = hypothesis.size(0)
        n_samples = premise.size(1)
        
        
        premise_char_vector = self.compute_character_embeddings(char_premise, n_timesteps_premise, n_samples, char_premise_mask)
        hypothesis_char_vector = self.compute_character_embeddings(char_hypothesis, n_timesteps_hypothesis, n_samples, char_hypothesis_mask)
        

        premise_word_emb = self.word_embeddings[premise.view(-1)].view(n_timesteps_premise, n_samples, self.dim_word)
        hypothesis_word_emb = self.word_embeddings[hypothesis.view(-1)].view(n_timesteps_hypothesis, n_samples, self.dim_word)
        
        hypothesis_emb = torch.cat((hypothesis_word_emb, hypothesis_char_vector), 2)
        hypothesis_emb = self.drop(hypothesis_emb)
        
        premise_emb = torch.cat((premise_word_emb, premise_char_vector), 2)
        premise_emb = self.drop(premise_emb)
    
        premise_seq, premise_rev_seq = self.sequence_encoder(premise_emb, premise_mask)
        hypothesis_seq, hypothesis_rev_seq = self.sequence_encoder(hypothesis_emb, hypothesis_mask)
        
        
        premise_comp = self.make_composite_vector(premise_seq, premise_rev_seq, premise_mask)
        hypothesis_comp = self.make_composite_vector(hypothesis_seq, hypothesis_rev_seq, hypothesis_mask)
        
    
        logit_0 = torch.cat((premise_comp, hypothesis_comp, torch.abs(premise_comp - hypothesis_comp), premise_comp * hypothesis_comp),1)
        logit = F.relu(self.Linear1(logit_0))
        logit = self.drop(logit)
        
        logit = torch.cat((logit_0, logit), 1)
        
        logit = F.relu(self.Linear2(logit))
        logit = self.drop(logit)
        
        logit = self.Linear3(logit)
        
        probs = F.softmax(logit)
        
        return probs
    
    def sequence_encoder(self, emb, mask):
        reverse_emb = self.reverseTensor(emb)
        reverse_mask = self.reverseTensor(mask)
        startl1 = time.time()
        #  LSTM1
        seq1 = self.LSTM1(emb, mask)
        seq_reverse1 = self.LSTM1_rev(reverse_emb, reverse_mask)
        
        inp_seq2 = torch.cat((seq1[0], self.reverseTensor(seq_reverse1[0])), len(seq1[0].size()) - 1)
        inp_seq2 = torch.cat((inp_seq2,emb),2)
        reverse_inp_seq2 = self.reverseTensor(inp_seq2)

        #  LSTM2
        seq2 = self.LSTM2(inp_seq2, mask)
        seq_reverse2 = self.LSTM2_rev(reverse_inp_seq2, reverse_mask)
        
        inp_seq3 = torch.cat((seq2[0], self.reverseTensor(seq_reverse2[0])), len(seq2[0].size()) - 1)
        inp_seq3 = torch.cat((inp_seq3,emb),2)
        reverse_inp_seq3 = self.reverseTensor(inp_seq3)


        #  LSTM3
        seq3 = self.LSTM3(inp_seq3, mask)
        seq_reverse3 = self.LSTM3_rev(reverse_inp_seq3, reverse_mask)
        
        return seq3,seq_reverse3
        
    def make_composite_vector(self, seq, seq_rev, mask):
        output = torch.cat((seq[0], self.reverseTensor(seq_rev[0])), len(seq[0].size()) - 1)
        
        gate = torch.cat((seq[1], self.reverseTensor(seq_rev[1])), len(seq[1].size()) - 1)
        gate = gate.norm(2, 2)

        mean = (output * mask[:, :, None]).sum(0) / (mask.sum(0)[:, None])
        maxi = (output * mask[:, :, None]).max(0)[0]
        gate_2 = (output * gate[:, :, None] * mask[:, :, None]).sum(0) / ((gate[:, :, None] * mask[:, :, None]).sum(0))
        rep = torch.cat((mean, maxi, gate_2),1)
        return rep

    def reverseTensor(self, tensor):
        idx = [i for i in range(tensor.size(0)-1, -1, -1)]
        if torch.cuda.is_available():
            idx = Variable(torch.LongTensor(idx).cuda())
        else:
            idx = Variable(torch.LongTensor(idx))
        inverted_tensor = tensor.index_select(0, idx)
        return inverted_tensor 
        
    def compute_character_embeddings(self, chars_word, n_timesteps, num_samples, char_mask):
        emb_char = self.Charemb(chars_word.view(-1)).view(n_timesteps, num_samples, chars_word.size(2), self.dim_char_emb)
        emb_char = emb_char * char_mask[:, :, :, None]
        emb_char_inp = emb_char.view(n_timesteps * num_samples, 1, chars_word.size(2), self.dim_char_emb)

        char_level_emb1 = self.apply_filter_and_get_char_embedding(self.conv1, emb_char_inp, num_samples, n_timesteps)
        char_level_emb2 = self.apply_filter_and_get_char_embedding(self.conv2, emb_char_inp, num_samples, n_timesteps)
        char_level_emb3 = self.apply_filter_and_get_char_embedding(self.conv3, emb_char_inp, num_samples, n_timesteps)
        
        emb_chars = [char_level_emb1, char_level_emb2, char_level_emb3]
        emb_char = torch.cat(emb_chars,2)
        return emb_char
    
    def apply_filter_and_get_char_embedding(self, conv, emb_char_inp, n_samples, n_timesteps):
        emb_char = conv(emb_char_inp)
        emb_char = F.relu(emb_char)
        emb_char = emb_char.view(n_timesteps * n_samples, self.char_nout, emb_char.size(2))
        emb_char = emb_char.max(2)[0]
        emb_char = emb_char.view(n_timesteps, n_samples, self.char_nout)
        return emb_char
        
        
    def init_weights(self, dim):
        initrange = 0.1
        self.init_convs(self.conv1, self.char_k_rows[0])
        self.init_convs(self.conv2, self.char_k_rows[1])
        self.init_convs(self.conv3, self.char_k_rows[2])
        
        self.Charemb.weight.data.uniform_(-initrange, initrange)
        self.Linear1.weight.data.uniform_(-initrange, initrange)
        self.Linear1.bias.data.fill_(0)
        self.Linear2.weight.data.uniform_(-initrange, initrange)
        self.Linear2.bias.data.fill_(0)
        self.Linear3.weight.data.uniform_(-initrange, initrange)
        self.Linear3.bias.data.fill_(0)


       
    def init_convs(self, conv, char_k_row):
        w_bound = math.sqrt(3 * char_k_row * self.char_k_cols)
        conv.weight.data.uniform_(-1.0/w_bound, 1.0/w_bound)
        conv.bias.data.fill_(0)
        

In [ ]:
train = TextIterator('word_sequence/premise_multinli_1.0_train.txt',
                     'word_sequence/hypothesis_multinli_1.0_train.txt',
                     'word_sequence/label_multinli_1.0_train.txt',
                     'word_sequence/vocab_cased.pkl',
                      n_words=num_words,
                      batch_size=batch_size)
valid = TextIterator('word_sequence/premise_multinli_1.0_dev_matched.txt',
                     'word_sequence/hypothesis_multinli_1.0_dev_matched.txt',
                     'word_sequence/label_multinli_1.0_dev_matched.txt',
                     'word_sequence/vocab_cased.pkl',
                      n_words=num_words,
                      batch_size=valid_batch_size,
                      shuffle=False)

test = TextIterator('word_sequence/premise_multinli_1.0_dev_mismatched.txt',
                    'word_sequence/hypothesis_multinli_1.0_dev_mismatched.txt',
                    'word_sequence/label_multinli_1.0_dev_mismatched.txt',
                    'word_sequence/vocab_cased.pkl',
                    n_words=num_words,
                    batch_size=valid_batch_size,
                    shuffle=False)

with open('word_sequence/vocab_cased.pkl', 'rb') as f:
    worddicts = pkl.load(f)
worddicts_r = dict()

for kk, vv in worddicts.items():
    worddicts_r[vv] = kk


In [ ]:
def pred_acc( iterator, worddicts_r):
    model.eval()
    valid_acc = 0
    n_done = 0
    num_times = 10

    for x1, x2, y in iterator:
        n_done += len(x1)
        tp1 = time.time()
        premise, premise_mask, char_premise, char_premise_mask, hypothesis, hypothesis_mask, char_hypothesis, char_hypothesis_mask, l = prepare_data(x1,x2,y,worddicts_r, max_len)
        tp2 = time.time()
        print(tp2-tp1)
        outputs = model(premise, char_premise, premise_mask, char_premise_mask, hypothesis, char_hypothesis, hypothesis_mask, char_hypothesis_mask,l)
        valid_acc += (outputs.max(1)[1] == l).sum().data[0]
    valid_acc = 1.0 * valid_acc / n_done
    return valid_acc

In [ ]:
model = NLI(dim_word, char_nout, dim_char_emb, 'data/glove.840B.300d.txt', worddicts, num_words, dim_hidden).cuda()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)  
print('training')
import time

loss = nn.CrossEntropyLoss() 
uidx =0
num_epochs = 10
validFreq = 5
eval_period = 1000
best_acc = 0.0 
model_path = 'ADAM_00001'
for epoch in range(num_epochs):
    for x1, x2, y in train:        
        model.train()
        uidx += 1
        #t1 = time.time()
        premise, premise_mask, char_premise, char_premise_mask, hypothesis, hypothesis_mask, char_hypothesis, char_hypothesis_mask, l = prepare_data(x1,x2,y,worddicts_r, max_len)
        #t2 = time.time()
        
        model.zero_grad()
        optimizer.zero_grad()

        #t3 = time.time()
        outputs = model(premise, char_premise, premise_mask, char_premise_mask, hypothesis, char_hypothesis, hypothesis_mask, char_hypothesis_mask,l)   
        
        #t4 = time.time()
        
        lossy = loss(outputs, l)
        #t5 = time.time()
        
        lossy.backward()
        #t6 = time.time()
        torch.nn.utils.clip_grad_norm(model.parameters(), 10.0)

        #t7 = time.time()
        optimizer.step()
        #t8 = time.time()
        #print('time to create data', t2-t1)
        #print('time to zero grad', t3-t2)
        #print('time to forward', t4-t3)
        #print('time to compute loss', t5-t4)
        #print('time to backward', t6-t5)
        #print('time to clip', t7-t6)
        #print('time to step', t8-t7)
        
        print(uidx,lossy.data[0])
        if uidx%eval_period == 0:
            print(outputs)
            valid_acc = pred_acc(valid, worddicts_r)
            print('Epoch ',epoch, ' Valid Accuracy = ', valid_acc)
            if(valid_acc>best_acc):
                torch.save(model, model_path)


In [ ]:
torch.save(model, 'SGD_0.01_3000iter')

In [ ]:
valid_acc = pred_acc(valid, worddicts_r)

In [ ]:
print(valid_acc)